In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('PUBG_Player_Statistics.csv')
data.info()

In [ ]:
data.head(5)

In [ ]:
concat = pd.concat([data.isnull().sum()],
          axis=1, 
          keys=['brakujące rekordy'])
print(concat.sum())

In [ ]:
described_columns=['solo_RideDistance','solo_RoadKills','solo_KillDeathRatio','duo_KillDeathRatio','solo_RoundsPlayed','solo_WinRatio','solo_LongestKill','duo_LongestKill','duo_HealsPg','duo_KillsPg','solo_WinRatio','squad_WinRatio','duo_DBNOs','duo_Revives']
print('player_name count:',data.player_name.count(),'\n')
for i in described_columns:
    print(data[i].describe().apply(lambda x: '{:2.2f}'.format(x, 'f')),'\n')

In [ ]:
KolmogorovSmirnov = [data.solo_KillDeathRatio,data.duo_KillDeathRatio,data.duo_HealsPg,data.duo_KillsPg] #sprawdzanie czy to rozkład normalny, potrzebna informacja do zmiannych używanych w teście różnic
for i in KolmogorovSmirnov: #dla każdej stat oddzielnie
    a,b = st.kstest(i,'norm')
    print(i.name,'stat: {:2.1f}'.format(a),'pvalue:',b)

In [ ]:
%matplotlib inline

## 1. Jaki jest związek między przebytą odległością przez gracza w pojeździe (solo_RideDistance), a ilością zabójstw poprzez potrącenie (solo_RoadKills) w trybie jednoosobowym?

In [ ]:
correlation, pvalue = st.pearsonr(data['solo_RideDistance'].values, data['solo_RoadKills'].values)

print('Poziom istotności: ', pvalue)
print('Poziom korelacji:', '{:2.2f}'.format(correlation))

## 7. Jaki jest związek między ilością wystąpień stanu DBNO (Down But Not Out) (duo_DBNOs) w trybie dwuosobowym, a ilością wskrzeszonych towarzyszy w trybie dwuosobowym (duo_Revives)?


In [ ]:
correlation, pvalue = st.pearsonr(data['duo_DBNOs'].values, data['duo_Revives'].values)

print('Poziom istotności: ', pvalue)
print('Poziom korelacji:', '{:2.2f}'.format(correlation))

## 3. Ze wszystkich graczy, którzy zagrali więcej niż 100 rund w trybie jednoosobowym (solo_RoundsPlayed), których dziesięciu (player_name) ma najwyższy procent wygranych w trybie jednoosobowym (solo_WinRatio)? W

In [ ]:
over_100_rounds = data.loc[data.solo_RoundsPlayed > 100, ['player_name', 'solo_WinRatio']]
over_100_rounds = over_100_rounds.sort_values('solo_WinRatio', ascending=False)

top10_over_100_rounds = over_100_rounds.head(10)
print(top10_over_100_rounds)

In [ ]:
plt.figure();
ax = top10_over_100_rounds.plot(x='player_name', y='solo_WinRatio', kind='bar', fontsize=10, rot=20, legend=False, figsize=(10,6))
plt.xlabel('player_name [-]', fontsize=11)
plt.ylabel('solo_WinRatio [-]', fontsize=11)
plt.title('Najwyższy procent wygranych w trybie jednoosobowym', fontsize=11)
plt.ylim([0, 40])
for p in ax.patches:
    ax.annotate('{:2.2f}%'.format(p.get_height()), (p.get_x() - 0.03, p.get_height() + 0.6))
plt.tight_layout()

plt.savefig('top_10_over_100_rounds.png')

## 4. W którym z trybów (jednoosobowy i dwuosobowy) gracze osiągają najwyższy średni wynik długości śmiertelnego strzału (LongestKill) W

In [ ]:
LongestKill_solo_vs_duo = data[['solo_LongestKill', 'duo_LongestKill']]
LongestKill_solo_vs_duo = LongestKill_solo_vs_duo.mean()
LongestKill_solo_vs_duo.apply(lambda x: '{:2.2f}'.format(x, 'f'))

In [ ]:
wykres = LongestKill_solo_vs_duo.plot.pie(autopct='%.2f % %', fontsize=11, figsize=(10, 10))
plt.legend(loc='upper left')
plt.title('Wykres porównujący średni wynik odległości śmiertelnego strzału dla tybu jednoosobowego i dwuosobowego')
plt.ylabel('')
plt.tight_layout()
plt.savefig('LongestKill_solo_vs_duo.png')

## 6. Jak wielu graczy, którzy mają współczynnik wygranych w trybie jednoosobowym poniżej 10 % (solo_WinRatio), mają współczynnik wygranych w trybie drużynowym powyżej 35% (squad_WinRatio). W

In [ ]:
WinRatio_solo_squad = data.loc[data.solo_WinRatio < 10 , ['solo_WinRatio','squad_WinRatio']]
WinRatio_solo_squad = WinRatio_solo_squad.loc[WinRatio_solo_squad.squad_WinRatio > 35 , ['solo_WinRatio','squad_WinRatio']]
WinRatio_solo_squad = WinRatio_solo_squad.count()
WinRatio_solo_squad['Reszta'] = data.solo_WinRatio.count()- WinRatio_solo_squad.solo_WinRatio
del WinRatio_solo_squad['squad_WinRatio']
WinRatio_solo_squad = WinRatio_solo_squad.rename({'solo_WinRatio': 'WinRatio_solo_squad_count'})
print(WinRatio_solo_squad)

In [ ]:
wykres = WinRatio_solo_squad.plot.pie(autopct='%.2f % %', fontsize=11, figsize=(9, 9), explode=[0.1,0], startangle=60)
plt.legend(loc='upper left')
plt.title('Wykres porównujący liczbę graczy spełniających wymagania w stosunku do całej reszty graczy')
plt.ylabel('')
plt.tight_layout()
plt.savefig('chosenones_vs_rest.png')

## 2. Czy jest różnica między wartością statystyki KDRatio w trybie solo (solo_KillDeathRatio), a w trybie duo?(duo_KillDeathRatio) W

In [ ]:
statistic, pvalue = st.mannwhitneyu(data.solo_KillDeathRatio,data.duo_KillDeathRatio)
print('Poziom istotności: ', pvalue)
print('Poziom różnic:', statistic)

In [ ]:
plt.figure(1 ,figsize=(10 ,10))
plt.tight_layout()
plt.xlabel('solo_KillDeathRatio [-]', fontsize=11)
plt.ylabel('duo_KillDeathRatio [-]', fontsize=11)
plt.title('Wykres zależności między KDRatio w trybie jedno i dwuosobowym', fontsize=11)
plt.scatter(data.solo_KillDeathRatio,data.duo_KillDeathRatio)
plt.tight_layout()
plt.savefig('KDRatio_solo_vs_duo.png')

## 5. Czy jest różnica między średnią liczbą uleczonych współtowarzyszy w trybie dwuosobowym (duo_HealsPg) a liczbą zabójstw w trybie dwuosobowym (duo_KillsPg)?

In [ ]:
statistic, pvalue = st.mannwhitneyu(data.duo_HealsPg,data.duo_KillsPg)
print('Poziom istotności: ', pvalue)
print('Poziom różnic:', statistic)